In [2]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

################CHANGE THESE AS NEEDED##############
TEMP=173
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs'
TARGET_SPECIES='N'
####################################################


os.chdir(TARGET_DIR) 
starttime=datetime.datetime.now()


print("Begin on {}".format(starttime.ctime()))
print("\nWorking dir:\t{}".format(os.getcwd())) 
print("""

TEMP:            {}
TARGET_DIR:      {}
TARGET_SPECIES:  {}
""".format(TEMP,TARGET_DIR,TARGET_SPECIES)
     )

print()




def theta_x(xi,yi,zi,xf,yf,zf):
    return np.arcsin(zf@yi)

def theta_y(xi,yi,zi,xf,yf,zf):
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)

def normalize(vector):
    return vector/np.linalg.norm(vector)

def ntheta_x(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    return np.arcsin(zf@yi)

def ntheta_y(xi,yi,zi,xf,yf,zf):
    n=normalize
    xi,yi,zi,xf,yf,zf=n(xi),n(yi),n(zi),n(xf),n(yf),n(zf)
    thetax = theta_x(xi,yi,zi,xf,yf,zf)
    return zf@xi/np.cos(thetax)



ls = os.listdir('.') 
efgfiles = [ thing for thing in ls if thing.startswith('efg') and thing.endswith('out') ]
numfiles= len(efgfiles) 
print("There are {} efg files in this directory.".format(numfiles))
print()



efgs=[] 
skipped_files=[]
for i in range(numfiles): 
    infile='efg.{}.out'.format(i) 
    try:
        efgs.append(efg.Efg(infile))
    except FileNotFoundError:
        skipped_files.append(infile)
        #print('skipped file {}'.format(infile))

if len(skipped_files) != 0:
    print('{} files were skipped.'.format(len(skipped_files)))
    print('between {} and {}'.format(skipped_files[0], skipped_files[-1]))
    print('display variable "skipped_files" to see all files not opened.')
print("{} efg files opened without error.".format(len(efgs)))
print()



print("The indices of {} atoms are:".format(TARGET_SPECIES)) 
target_atom_labels=[] 
for label in efgs[0].atom_labels: 
    if TARGET_SPECIES in label: 
        new_index = efgs[0].atom_labels.index(label)+1 
        target_atom_labels.append(new_index) 
        sys.stdout.write(str(new_index)+' ') 

print()





av_theta_xs = [] 
av_theta_ys = [] 
av_theta_y2s = [] 
av_theta_x2s = [] 
cq_coef0s = [] 
cq_coef1s = [] 
eta_coefs = [] 
cq0s = [] 
cqprime0s = [] 
cqprime1s = [] 
eta0s = [] 
etaprimes = [] 
fq0s = [] 
fqprimes = []

for atnum in target_atom_labels: 
    efg0 = efgs[0] 
    specie=atnum 
    k=specie 
    print("\nWorking on atomic specie:\t{}".format(specie)) 
    
    
    this_nucleus=efg0.atom(k) 
    cq0=this_nucleus.cq 
    cq0s.append(cq0) 
    eta0=this_nucleus.eta
    eta0s.append(eta0)
    
    
    xi=efg0.atom(k).x
    yi=efg0.atom(k).y
    zi=efg0.atom(k).z



    theta_x_vec=[]
    theta_y_vec=[]


    print("Fetching the angles in file efg.{}.out")
    for i in range(1,901):
        try:
            nucleus=efgs[i].atom(k)
            nuc=nucleus
            xf,yf,zf=nuc.x,nuc.y,nuc.z
            args=(xi,yi,zi,xf,yf,zf)
            #print("thetaX{}\t".format(i),"\t", theta_x(xi,yi,zi,xf,yf,zf),"\t", "thetaY{}\t".format(i), theta_y(xi,yi,zi,xf,yf,zf))
            theta_x_vec.append(theta_x(*args))
            theta_y_vec.append(theta_y(*args))
            sys.stdout.write("{} \r".format(i))
        except IndexError:
            sys.stdout.write("skipped:{}\n".format(i))
        
  
                 # theta_x squareds and theta_y squareds
    theta_x2_vec=[angle**2 for angle in theta_x_vec]
    theta_y2_vec=[angle**2 for angle in theta_y_vec]

    av=np.average
    av_theta_x  = av(theta_x_vec)
    av_theta_y  = av(theta_y_vec)
    av_theta_x2 = av(theta_x2_vec)
    av_theta_y2 = av(theta_y2_vec)

    av_theta_xs.append(av_theta_x)
    av_theta_ys.append(av_theta_y)
    av_theta_x2s.append(av_theta_x2)
    av_theta_y2s.append(av_theta_y2)

    print("\nReport for atomic specie:\t{}".format(k))
    print("Working Dir:\t{}".format(os.getcwd()))
    print("<tx>:\t",av_theta_x,"\t<ty>:\t",   av_theta_y,"\n",
      "<tx^2>\t:",av_theta_x2,"\t<ty^2>:", av_theta_y2,"\n")

    print("cq0=\t{}".format(cq0))
    print("eta0=\t{}".format(eta0))

    cq_coef0 = 1 - 3/2*(av_theta_x2 + av_theta_y2)
    cq_coef0s.append(cq_coef0)
    cq_coef1 = 1 - 3/2*(av_theta_x2 + av_theta_y2 - 1/2*eta0*(av_theta_x2 - av_theta_y2))
    cq_coef1s.append(cq_coef1)

    
    
    print("==================================")
    print("target:\t"+TARGET_DIR)
    print("cwd:\t"+os.getcwd())
    print("T={}K, nuclear site: {}  =".format(TEMP,specie))
    print("==================================")


    print("1-3/2(<tx^2>+<ty^2>)\t\t={}".format(cq_coef0))
    print("1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)\t={}".format(cq_coef1))
    print()


    cqprime0=cq0*cq_coef0
    cqprime0s.append(cqprime0)
    cqprime1=cq0*cq_coef1
    cqprime1s.append(cqprime1)


    print("Cq0:\t\t{}".format(cq0))
    print("Cq' from coefficient 1-3/2(<>+<>):\t\t{}".format(cqprime0))
    print("Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):\t{}".format(cqprime1))
    nuclear_spin=1
    freq_function=nqr.f1
    print(
    "Computing NQR frequencies for spin:\t{}\nusing function {}\tand simple coefficient".format(nuclear_spin, freq_function)
    )
    print("WARNING:\t eta not adjusted.  Using eta0.")
    fq=freq_function(cqprime0,eta0)
    fq0s.append(fq)
    print("fqs={}".format(freq_function(cqprime0, eta0)))
    print()
    print()

print("##################################"*2)
print("##################################"*2)
print()
print("==================================")
print("          FINAL REPORT T={}K      ".format(TEMP))
print("cwd:\t{}".format(os.getcwd()))
print("==================================")
print("""
target_atom_labels=\t{}\n
av_theta_xs=\t{}\n    
av_theta_ys=\t{}\n  
av_theta_y2s=\t{}\n
av_theta_x2s=\t{}\n 
cq_coef0s=\t{}\n
cq_coef1s=\t{}\n
eta_coefs=\t{}\n
cq0s=\t{}\n
cqprime0s=\t{}\n
cqprime1s=\t{}\n 
eta0s=\t{}\n
etaprimes=\t{}\n
fq0s=\t{}\n 
fqprimes=\t{}\n


    """.format(
    target_atom_labels,
    av_theta_xs,
    av_theta_ys, 
    av_theta_y2s, 
    av_theta_x2s, 
    cq_coef0s, 
    cq_coef1s, 
    eta_coefs, 
    cq0s, 
    cqprime0s, 
    cqprime1s, 
    eta0s, 
    etaprimes, 
    fq0s, 
    fqprimes))





print("cq_coef0s\n---------")
for item in target_atom_labels:
    print(item, cq_coef0s[target_atom_labels.index(item)])
print()


print("cq_coef1s\n---------")
for item in target_atom_labels:
    print(item, cq_coef1s[target_atom_labels.index(item)])
print()


print("cq0s\n---------")
for item in target_atom_labels:
    print(item, cq0s[target_atom_labels.index(item)])
print()


print("cqprime0s\n---------")
for item in target_atom_labels:
    print(item, cqprime0s[target_atom_labels.index(item)])
print()


print("cqprime1s\n---------")
for item in target_atom_labels:
    print(item, cqprime1s[target_atom_labels.index(item)])
print()


print("eta0s\n---------")
for item in target_atom_labels:
    print(item, eta0s[target_atom_labels.index(item)])
print()





endtime=datetime.datetime.now()
runtime=endtime-starttime
runtime_s=runtime.total_seconds()

print("DONE\truntime is {} seconds".format(str(runtime_s)))
print(endtime.ctime())
print("Goodbye.")
print()




Begin on Mon Feb 19 15:12:04 2018

Working dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs


TEMP:            173
TARGET_DIR:      /Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
TARGET_SPECIES:  N


There are 901 efg files in this directory.

901 efg files opened without error.

The indices of N atoms are:
1 3 4 6 23 25 26 28 37 39 40 42 51 53 54 56 

Working on atomic specie:	1
Fetching the angles in file efg.{}.out
skipped:795
skipped:796
skipped:797
skipped:798
skipped:799
skipped:800
skipped:858
skipped:895
skipped:896
skipped:897
skipped:898
skipped:899
skipped:900

Report for atomic specie:	1
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
<tx>:	 -0.0125043999376 	<ty>:	 0.00207032675989 
 <tx^2>	: 0.00272837055425 	<ty^2>: 0.00267572054636 

cq0=	-6.214
eta0=	0.50991
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
T=173K, nuclear s

Fetching the angles in file efg.{}.out
skipped:795
skipped:796
skipped:797
skipped:798
skipped:799
skipped:800
skipped:858
skipped:895
skipped:896
skipped:897
skipped:898
skipped:899
skipped:900

Report for atomic specie:	28
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
<tx>:	 0.0134821432793 	<ty>:	 0.00024360136516 
 <tx^2>	: 0.00180982646245 	<ty^2>: 0.00315272688893 

cq0=	-6.214
eta0=	0.50991
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
T=173K, nuclear site: 28  =
1-3/2(<tx^2>+<ty^2>)		=0.9925561699729345
1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.9920426012055837

Cq0:		-6.214
Cq' from coefficient 1-3/2(<>+<>):		-6.167744040211815
Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):	-6.164552723891497
Computing NQR frequencies for spin:	1
using function <function f1 at 0x106b8be18>	and simple coefficient
fqs=[-1.57249718 -3.83955944 -5.41205662]



Working on 

 <tx^2>	: 0.0197676637516 	<ty^2>: 0.0079160765375 

cq0=	-0.7282
eta0=	0.62328
target:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
cwd:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
T=173K, nuclear site: 54  =
1-3/2(<tx^2>+<ty^2>)		=0.9584743895663793
1-3/2(<tx^2>+<ty^2>-1/2eta0*(<tx^2>-<ty^2>)	=0.9640145325254711

Cq0:		-0.7282
Cq' from coefficient 1-3/2(<>+<>):		-0.6979610504822373
Cq' from coefficient 1-3/2(<>+<>)+1/2eta(<>-<>)):	-0.701995382585048
Computing NQR frequencies for spin:	1
using function <function f1 at 0x106b8be18>	and simple coefficient
fqs=[-0.21751258 -0.4147145  -0.63222708]



Working on atomic specie:	56
Fetching the angles in file efg.{}.out
skipped:795
skipped:796
skipped:797
skipped:798
skipped:799
skipped:800
skipped:858
skipped:895
skipped:896
skipped:897
skipped:898
skipped:899
skipped:900

Report for atomic specie:	56
Working Dir:	/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933/scfs
<tx>:	 0.0171169894165 	<ty>:	

In [1]:
import efg
import md
import sys
import numpy as np
import os
import nqr_parser as nqr
import datetime

target_atom_labels=	[1, 3, 4, 6, 23, 25, 26, 28, 37, 39, 40, 42, 51, 53, 54, 56]

av_theta_xs=	[-0.012504399937636987, -0.0099352678689437028, 0.014932593426693005, -0.0045704317946508068, -0.040404880333382749, 0.00040706822891480477, 0.0048496241581253959, 0.013482143279311543, 0.0142796260397403, 0.022555618661973309, -0.0025222925818466474, -0.015298061989071972, -0.048548803334872451, -0.004497221332187786, 0.0044650811140973176, 0.017116989416511429]
    
av_theta_ys=	[0.0020703267598879485, -0.0049462357557854639, -0.015341161521357328, -0.0044760793675660115, -0.0084394494450360201, -0.00011108444866381009, 0.0036876377831817583, 0.00024360136516019066, 0.0027923609443847646, -0.011924435362545853, -0.015329279357869065, 0.0039501331865259195, 0.0036220238458252447, 0.00059830556224673033, -0.0054578033255322242, 0.0011047162490641742]
  
av_theta_y2s=	[0.0026757205463610332, 0.011688216537888346, 0.0023551662313563769, 0.0075825599499999225, 0.010690476912126179, 0.0023165579991033043, 0.0099426120963245906, 0.0031527268889297764, 0.012720227544955748, 0.0020884501284199278, 0.017475668917717443, 0.0027868831785580151, 0.0054359799731837728, 0.0025849151458823519, 0.0079160765375023839, 0.0031361961505231434]

av_theta_x2s=	[0.0027283705542470674, 0.025672080959331028, 0.0025730353283679366, 0.024656389799564308, 0.032762571570455465, 0.0041645519056369073, 0.038392218169981993, 0.0018098264624472338, 0.02724001159027261, 0.0031829607406058035, 0.034058108170478545, 0.0038536053392314146, 0.031787029242231424, 0.002446536013856537, 0.019767663751578082, 0.0029118568040506347]
 
cq_coef0s=	[0.9918938633490878, 0.94395955375417095, 0.99260769766041357, 0.95164157537565364, 0.93482042727612757, 0.99027833514288965, 0.92749775460054007, 0.9925561699729345, 0.94005964129715747, 0.99209288369646143, 0.92269933436770601, 0.99003926722331581, 0.94416548617687723, 0.99245282326039164, 0.95847438956637931, 0.99092792056813939]

cq_coef1s=	[0.99191399842322869, 0.95049624125865217, 0.99268490665568521, 0.95884293264502829, 0.94413043680301079, 0.99093321798351752, 0.94079680745573202, 0.99204260120558374, 0.94618375951233114, 0.99248075910348432, 0.93045071268421298, 0.99044721644602762, 0.95528035875856154, 0.99240378515548, 0.96401453252547109, 0.9908421259110195]

eta_coefs=	[]

cq0s=	[-6.214, -0.7282, -6.08, -0.7971, -0.7971, -6.08, -0.7282, -6.214, -0.7971, -6.08, -0.7282, -6.214, -0.7971, -6.08, -0.7282, -6.214]

cqprime0s=	[-6.1636284668512324, -0.68739134704378724, -6.0350548017753143, -0.75855349973193353, -0.74514536258180131, -6.0208922776687688, -0.67540386490011328, -6.1677440402118151, -0.74932154007796425, -6.0319247328744856, -0.67190965528656343, -6.1521040065256845, -0.75259430903158886, -6.0341131654231814, -0.69796105048223733, -6.157626098410419]

cqprime1s=	[-6.1637535862019437, -0.69215136288455048, -6.0355242324665666, -0.76429370161135213, -0.75256637117567993, -6.0248739653397863, -0.68508823518926398, -6.1645527238914974, -0.75420307470727921, -6.0342830153491844, -0.67755420897664387, -6.1546390029956157, -0.76145397396644943, -6.0338150137453184, -0.70199538258504801, -6.1570929704110755]
 
eta0s=	[0.50991, 0.62326, 0.47251, 0.56237, 0.5624, 0.4725, 0.62328, 0.50991, 0.56237, 0.47251, 0.62326, 0.50991, 0.5624, 0.4725, 0.62328, 0.50991]

etaprimes=	[]

fq0s=  [np.array([-1.5714479, -3.8369974, -5.4084453]),np.array([-0.21421177, -0.40843763, -0.62264939]),np.array([-1.42581187, -3.81338517, -5.23919704]),np.array([-0.21329387, -0.46226819, -0.67556206]),np.array([-0.20953488, -0.45409158, -0.66362646]),np.array([-1.4224358 , -3.80445131, -5.22688711]),np.array([-0.21048286, -0.40131147, -0.61179433]),np.array([-1.57249718, -3.83955944, -5.41205662]),np.array([-0.21069798, -0.45664217, -0.66734014]),np.array([-1.42507238, -3.81140736, -5.23647974]),np.array([-0.20938721, -0.39923864, -0.60862584]),np.array([-1.56850968, -3.82982317, -5.39833284]),np.array([-0.21162952, -0.45863097, -0.67026049]),np.array([-1.42555924, -3.81280526, -5.23836449]),np.array([-0.21751258, -0.4147145 , -0.63222708]),np.array([-1.56991756, -3.83326079, -5.40317835])]

fqprimes=	[]

TEMP=173
TARGET_DIR='/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933'
os.chdir(TARGET_DIR)
os.getcwd()
x=md.Md('md.in','md.out')
vol=x.cell_volume
print(TARGET_DIR,"\n","-"*len(TARGET_DIR),"\n")

def get_inds(target, inds):
    result = []
    for ind in inds:
        result.append(target[ind])
    return result

axial_inds = [ 0,  7, 11, 15]
equit_inds = [ 2,  5,  9, 13]
ax         = axial_inds
eq         = equit_inds

axial_cq0s = get_inds(cq0s, ax)
axial_eta0s= get_inds(eta0s, ax)
axial_k0s  = get_inds(cq_coef0s, ax)
axial_k1s  = get_inds(cq_coef1s, ax)
axial_cq   = np.average(get_inds(cq0s, ax))
axial_eta  = np.average(get_inds(eta0s, ax))
axial_k0   = np.average(get_inds(cq_coef0s, ax))
axial_k1   = np.average(get_inds(cq_coef1s, ax))


equit_cq0s = get_inds(cq0s, eq)
equit_eta0s= get_inds(eta0s, eq)
equit_k0s  = get_inds(cq_coef0s, eq)
equit_k1s  = get_inds(cq_coef1s, eq)
equit_cq   = np.average(get_inds(cq0s, eq))
equit_eta  = np.average(get_inds(eta0s, eq))
equit_k0   = np.average(get_inds(cq_coef0s, eq))
equit_k1   = np.average(get_inds(cq_coef1s, eq))



print("cell volume:", vol,"\n")
print("AXIAL","\n","-"*len(TARGET_DIR))
print("avg Cq0:", np.average(axial_cq0s),"\t",axial_cq)
print("avg eta0:", np.average(axial_eta0s),"\t",axial_eta)
print("avg coef0:",np.average(axial_k0s),"\t",axial_k0)
print("avg coef1:",np.average(axial_k1s),"\t",axial_k1)
print("using coef0:",axial_k0*axial_cq)
print("using coef1:",axial_k1*axial_cq)


ffunc=nqr.f1
fraw=ffunc(axial_cq,axial_eta)
f0=ffunc(axial_k0*axial_cq,axial_eta)
f1=ffunc(axial_k1*axial_cq,axial_eta)
exptcq=6.029
expteta=0.5205
exptfq=[5.306,       3.737]
print("expt cq:\t", exptcq)
print("raw freq:\t",list(reversed(fraw))[:-1])

print("expt fq:\t", exptfq)
print("adjusted f0s:\t",list(reversed(f0))[:-1])
print("adjusted f1s:\t",list(reversed(f1))[:-1])
print("\n\n")

print("EQUITORIAL","\n","-"*len(TARGET_DIR))
print("avg Cq0:", np.average(equit_cq0s),"\t",equit_cq)
print("avg eta0:", np.average(equit_eta0s),"\t",equit_eta)
print("avg coef0:",np.average(equit_k0s),"\t",equit_k0)
print("avg coef1:",np.average(equit_k1s),"\t",equit_k1)
print("using coef0:",equit_k0*equit_cq)
print("using coef1:",equit_k1*equit_cq)


ffunc=nqr.f1
fraw=ffunc(equit_cq,equit_eta)
f0=ffunc(equit_k0*equit_cq,equit_eta)
f1=ffunc(equit_k1*equit_cq,equit_eta)
exptcq=5.795
expteta=0.4980
exptfq=[5.068,       3.625]
print("expt cq:\t", exptcq)
print("raw freq:\t",list(reversed(fraw))[:-1])

print("expt fq:\t", exptfq)
print("adjusted f0s:\t",list(reversed(f0))[:-1])
print("adjusted f1s:\t",list(reversed(f1))[:-1])

/Users/altoidnerd/Desktop/nitroamines3/hmx/dt80N6/792933 
 -------------------------------------------------------- 

cell volume: 508.914063262 

AXIAL 
 --------------------------------------------------------
avg Cq0: -6.214 	 -6.214
avg eta0: 0.50991 	 0.50991
avg coef0: 0.991354305278 	 0.991354305278
avg coef1: 0.991311485496 	 0.991311485496
using coef0: -6.160275653
using coef1: -6.16000957088
expt cq:	 6.029
raw freq:	 [-5.4526451850000006, -3.8683548150000009]
expt fq:	 [5.306, 3.737]
adjusted f0s:	 [-5.4055032793051208, -3.8349102001945603]
adjusted f1s:	 [-5.405269798227498, -3.8347445580850539]



EQUITORIAL 
 --------------------------------------------------------
avg Cq0: -6.08 	 -6.08
avg eta0: 0.472505 	 0.472505
avg coef0: 0.99185793494 	 0.99185793494
avg coef1: 0.992125667225 	 0.992125667225
using coef0: -6.03049624444
using coef1: -6.03212405673
expt cq:	 5.795
raw freq:	 [-5.2782076000000009, -3.8417924000000006]
expt fq:	 [5.068, 3.625]
adjusted f0s:	 [-5.23523